In [12]:
import pandas as pd
import pickle
import metrics 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

ModuleNotFoundError: No module named 'metrics'

In [2]:
df = pd.read_csv('../../Dataset/Cardiovascular_Disease_Dataset.csv')

# separamos el target del resto de las variables
X, y = df.iloc[:,1:-1], df[['target']]
columns_x = X.columns

# escalamos las variables
scaler = StandardScaler()
X = scaler.fit_transform(X)
X = pd.DataFrame(X, columns=columns_x)


In [3]:
X

,age,gender,chestpain,restingBP,serumcholestrol,fastingbloodsugar,restingrelectro,maxheartrate,exerciseangia,oldpeak,slope,noofmajorvessels
0,0.210464,0.554247,1.070663,0.642833,-2.352717,-0.648425,0.327384,0.044567,-0.996008,1.507245,1.455350,1.819678
1,-0.517591,0.554247,-1.028677,-1.928098,-0.622817,-0.648425,0.327384,-0.891840,-0.996008,0.576955,-0.538280,-0.227204
2,-0.013553,0.554247,1.070663,-0.625938,-1.280028,-0.648425,-0.971760,1.654017,1.004008,1.332816,-0.538280,-1.250645
3,-0.349578,0.554247,-1.028677,-0.458995,-0.124243,1.542199,0.327384,0.220143,-0.996008,0.286239,0.458535,0.796237
4,-1.021629,0.554247,0.020993,1.577717,-2.352717,-0.648425,1.626528,-0.277323,-0.996008,1.507245,1.455350,0.796237
...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.069557,0.554247,1.070663,-0.425606,0.283681,-0.648425,1.626528,1.098025,1.004008,1.681675,0.458535,0.796237
996,-0.125561,0.554247,2.120333,-0.292051,-0.403747,1.542199,0.327384,-1.389306,1.004008,1.739818,-0.538280,-1.250645
997,1.106531,0.554247,-1.028677,0.142002,0.925784,1.542199,-0.971760,1.478441,-0.996008,-0.760338,1.455350,-0.227204
998,-0.237570,0.554247,0.020993,1.143664,0.797363,-0.648425,0.327384,-0.833315,1.004008,1.856104,1.455350,0.796237


In [4]:
# concatenamos las variables escaladas con el target
df_scaled = pd.concat([X, y], axis=1)

# Split the data into train and test sets.
train_df, test_df = train_test_split(df_scaled, test_size=0.2, random_state=42)

# Save the train and test sets.
train_df.to_csv('../../Dataset/train.csv', index=False, header=True)
test_df.to_csv('../../Dataset/test.csv', index=False, header=True)

In [14]:
# read the data
df = pd.read_csv('../../Dataset/train.csv')

# split the data into X and y
X, y = df.iloc[:, :-1], df[['target']]

# train the model
model = SGDClassifier(random_state=42)
model.fit(X, y)

# evaluate the model
y_pred = model.predict(X)
accuracy = accuracy_score(y, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y, y_pred, average="macro")

# write the model
pickle.dump(model, open("../../Model/model.pkl", "wb"))



d:\Users\ecdia\miniconda3\envs\env_prueba\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [17]:
"""Trains a model on tabular data."""
import pandas as pd
import joblib
import os
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import (accuracy_score, precision_recall_curve,
                                roc_auc_score)
from sklearn.model_selection import (RandomizedSearchCV, StratifiedKFold,
                                        train_test_split)
from sklearn.preprocessing import LabelEncoder

# read the data
df = pd.read_csv('../../Dataset/Cardiovascular_Disease_Dataset.csv')

# split the data into X and y
X, y = df.iloc[:, :-1], df[['target']]

# Assuming X and y are predefined
X_train, X_test, y_train, y_test = train_test_split(X, y)

params = {
    "alpha": [0.0001, 0.001, 0.01, 0.1],
    "penalty": ["l2", "l1", "elasticnet"],
    "loss": ["hinge", "log", "modified_huber"],
    "max_iter": [1000, 2000, 3000]
}

sgd_model = SGDClassifier(random_state=42)

folds = 3
param_comb = 10

skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)

random_search = RandomizedSearchCV(
    sgd_model,
    param_distributions=params,
    n_iter=param_comb,
    scoring="precision",
    n_jobs=4,
    cv=skf.split(X_train, y_train),
    verbose=4,
    random_state=42,
)

random_search.fit(X_train, y_train)
sgd_model_best = random_search.best_estimator_
predictions = sgd_model_best.predict(X_test)
score = accuracy_score(y_test, predictions)
auc = roc_auc_score(y_test, predictions)
_ = precision_recall_curve(y_test, predictions)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


d:\Users\ecdia\miniconda3\envs\env_prueba\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
